In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns',500)


In [2]:
df = pd.read_csv('data/pbp_data_mvp.csv')
df['Date'] = pd.to_datetime(df["Date"])

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (14,16,86,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
team_list = df.home_team.unique()
year_list = df.Date.dt.year.unique()

These are rows that do not have any impact on the game in a way that can be useful for prediction. 

In [4]:
cols_to_drop = ['Unnamed: 0','forfeit_info','lf_ump_id','rf_ump_id','protest_info',
                    'date_game_completed','additional_info','save_pitch_id',
                    'game_win_rbi_batter_id','game_in_series','away_catch_interference',
                'home_catch_interference','away_pitch_balks','home_pitch_balks']
drop_cols_before_modeling = ['day_of_week','away_league',
                            'away_team_game_number','home_league',
                            'home_team_game_number','day_or_night','park_id','attendance',
                            'time_of_game','away_line_scores','home_line_scores','year','id',
                            'outcome']

df.drop(columns=cols_to_drop,inplace=True)
df.drop(columns=drop_cols_before_modeling,inplace=True)
df.drop(df.loc[:,'hb_ump_id':'acquisition_info'],axis=1,inplace=True)

Now I must get rid of all statistics that are not able to be averaged to create cumulative statistics over a year.

In [5]:
df.head(1)

,Date,away_team,home_team,away_team_score,home_team_score,number_of_outs,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_OBP,away_OBP,home_AVG,away_AVG,home_singles,away_singles,home_SLG,away_SLG,home_BABIP,away_BABIP,home_ISO,away_ISO,home_PASO,away_PASO,home_total_bases,away_total_bases,home_runs_created,away_runs_created,home_wOBA,away_wOBA,game_number_of_season,home_won_last_game,away_won_last_game,target,away_outcome,home_outcome
0,2000-03-30,NYN,CHN,5,1,66,37,6,2,0,1,5,1,1,1,8,0,5,1,0,0,10,5,0,0,0,33,14,2,0,2,0,36,5,0,0,0,0,2,0,0,6,1,9,0,0,2,10,7,5,5,0,33,14,0,0,0,0,0.261905,0.319149,0.138889,0.162162,5,3,0.138889,0.297297,0.185185,0.15625,0.0,0.135135,4.0,7.4,5,11,1.309524,3.422222,0.209512,0.288298,1,0,0,0,1,0


In [69]:
drop_for_cummean = ['number_of_outs','target','game_number_of_season',
                   'away_won_last_game','home_won_last_game','game_number_of_season']
df.drop(columns=drop_for_cummean,inplace=True)

In [99]:
# making dummy rows for use in putting games into year/team buckets
df['home_date'] = df.Date
df['away_date'] = df.Date

In [232]:
# initialize dict so that we can aggregate the stats of each team per year
# final dict is where we are going to be adding the aggregated stats
stat_dict = {}
final_dict = {}
for year in year_list:
    stat_dict[year] = {}
    final_dict[year] = {}
    for team in team_list:
        stat_dict[year][team] = []
        final_dict[year][team] = []

In [233]:
# this is putting games into the correct year/team combo
# we need both home and away games bc away stats obviously affect stats
# of the team
for year in year_list:
    for team in team_list:
        # home game stats
        for game in df[(df.Date.dt.year == year)&(df.home_team == team)].filter(regex='home').values.tolist():
            stat_dict[year][team].append(np.array(game))
        # away game stats
        for game in df[(df.Date.dt.year == year)&(df.away_team == team)].filter(regex='away').values.tolist():
            stat_dict[year][team].append(np.array(game))
            

In [234]:
from operator import itemgetter

In [235]:
# have to sort each year/team array so that they are in the correct date order
for year in year_list:
    for team in team_list:
        stat_dict[year][team] = sorted(stat_dict[year][team],key=itemgetter(-1))
        #print((stat_dict[year][team]))

In [236]:
# putting date in the front to make it easier to work with
for year in year_list:
    for team in team_list:
        year_team_stats = []
        for game in stat_dict[year][team]:
            #print(game[-1])
            #print(game[:-1])
            #print(np.insert(game[:-1],0,game[-1]))
            year_team_stats.append(np.insert(game[:-1],0,game[-1]))
        stat_dict[year][team] = year_team_stats

In [237]:
# this is aggregating the stats per year
# so each game is the mean of all stats of that game and all previous
for year in year_list:
    for team in team_list:
        aggregate_stats = np.array(stat_dict[year][team][0][2:])
        for game in stat_dict[year][team]:
            header_info = game[0:2]
            contents = game[2:]
            aggregate_stats = (contents+aggregate_stats)/2
            final_dict[year][team].append(np.concatenate((header_info,aggregate_stats)))

In [238]:
agg_df = pd.DataFrame()

In [239]:
# creating the aggregate stats array
agg_array = []
for year in year_list:
    for team in team_list:
        for game in final_dict[year][team]:
            agg_array.append(game)

In [240]:
# making the array of all stats
agg_df = pd.DataFrame(agg_array)

In [267]:
away_data_column_names = list(df.filter(regex='away').columns)[1:-1] # gets rid of team name and date
home_data_column_names = list(df.filter(regex='home').columns)[1:-1] # gets rid of team name and date

In [364]:
game_basic_info_df = df.iloc[:,0:3]
for stat in away_data_column_names:
    game_basic_info_df[stat] = ''
for stat in home_data_column_names:
    game_basic_info_df[stat] = ''

In [379]:
final_matrix_for_df = []

In [382]:
for index,row in game_basic_info_df.iterrows():
    # append home team aggregate stats
    home_data = agg_df[(agg_df[0] == row.Date)&(agg_df[1]==row.home_team)].iloc[:,2:].to_numpy()
    # append away team aggregate stats
    away_data = agg_df[(agg_df[0] == row.Date)&(agg_df[1]==row.away_team)].iloc[:,2:].to_numpy()
    # create the full row
    full_row = np.concatenate((home_data[0],away_data[0]))
    temp_array = [row.Date,row.home_team,row.away_team]
    for individual_stat in full_row:
        temp_array.append(individual_stat)
    final_matrix_for_df.append(temp_array)
    #game_basic_info_df[(game_basic_info_df.home_team == row.home_team)&(game_basic_info_df.Date == row.Date)].iloc[:,3:] = full_row
    #row.iloc[3:] = full_row

In [372]:
game_basic_info_df[(game_basic_info_df.home_team == row.home_team)&(game_basic_info_df.Date == row.Date)].iloc[:,3:]

,away_team_score,away_at_bats,away_hits,away_doubles,away_triples,away_hrs,away_rbi,away_sh,away_sf,away_hbp,away_walk,away_int_walk,away_so,away_sb,away_cs,away_gidp,away_left_on_base,away_pitchers_used,away_pitch_earned_runs,away_team_earned_runs,away_pitch_wild_pitches,away_def_putouts,away_def_assists,away_def_errors,away_def_passed_balls,away_def_double_plays,away_def_triple_plays,away_OBP,away_AVG,away_singles,away_SLG,away_BABIP,away_ISO,away_PASO,away_total_bases,away_runs_created,away_wOBA,away_outcome,home_team_score,home_at_bats,home_hits,home_doubles,home_triples,home_hrs,home_rbi,home_sh,home_sf,home_hbp,home_walk,home_int_walk,home_so,home_sb,home_cs,home_gidp,home_left_on_base,home_pitchers_used,home_pitch_earned_runs,home_team_earned_runs,home_pitch_wild_pitches,home_def_putouts,home_def_assists,home_def_errors,home_def_passed_balls,home_def_double_plays,home_def_triple_plays,home_OBP,home_AVG,home_singles,home_SLG,home_BABIP,home_ISO,home_PASO,home_total_bases,home_runs_created,home_wOBA,home_outcome
48567,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [383]:
final_final_df = pd.DataFrame(final_matrix_for_df)

In [391]:
# so this isnt actually cumulative its just total so far + this row / 2 which isnt how math works
final_final_df[((final_final_df[1]=='ATL')|(final_final_df[2]=='ATL'))&(final_final_df[0].dt.year == 2000)].head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
3,2000-04-03,ATL,COL,2.000000,30.000000,7.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,6.000000,1.000000,0.000000,1.000000,6.000000,2.000000,0.000000,0.000000,0.000000,27.000000,12.000000,0.000000,0.000000,1.000000,0.0,0.281250,0.233333,5.000000,0.433333,0.227273,0.200000,5.000000,13.000000,3.354839,0.314375,1.000000,0.000000,31.000000,6.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,7.000000,0.000000,0.000000,1.000000,7.000000,3.000000,2.000000,2.000000,1.000000,24.000000,10.000000,0.000000,0.000000,1.000000,0.0,0.242424,0.193548,4.000000,0.258065,0.250000,0.064516,4.428571,8.000000,1.939394,0.241290,0.000000
19,2000-04-04,ATL,COL,2.500000,31.500000,6.500000,1.000000,0.000000,1.500000,2.500000,0.000000,0.000000,1.000000,2.500000,1.000000,5.000000,1.000000,0.500000,0.500000,7.000000,3.500000,2.500000,2.500000,0.000000,27.000000,11.000000,0.000000,0.000000,0.500000,0.0,0.285362,0.207576,4.000000,0.383333,0.202922,0.175758,6.625000,12.000000,3.163906,0.307049,0.500000,2.500000,33.500000,8.500000,3.000000,0.500000,0.000000,2.500000,1.000000,0.000000,0.000000,3.500000,2.000000,6.500000,1.000000,0.500000,0.500000,8.500000,3.000000,2.500000,2.500000,1.000000,25.500000,14.000000,1.000000,0.000000,0.500000,0.0,0.316334,0.249552,5.000000,0.365143,0.308333,0.115591,5.214286,12.500000,4.286770,0.319235,0.500000
28,2000-04-05,ATL,COL,5.750000,33.750000,9.250000,1.000000,0.000000,1.250000,4.250000,0.000000,0.000000,1.500000,3.750000,0.500000,4.500000,0.500000,0.250000,0.750000,8.500000,3.750000,3.750000,3.750000,0.000000,27.000000,11.500000,1.000000,0.000000,0.750000,0.0,0.363611,0.270455,7.000000,0.413889,0.278880,0.143434,7.812500,14.000000,4.899026,0.353059,0.750000,4.250000,33.750000,8.750000,1.500000,0.750000,1.000000,3.250000,2.000000,0.000000,0.000000,2.250000,1.000000,5.250000,0.500000,0.250000,0.750000,6.750000,3.500000,4.250000,4.250000,0.500000,24.750000,11.000000,1.500000,0.000000,0.750000,0.0,0.301024,0.257129,5.500000,0.432572,0.279167,0.175443,6.857143,14.750000,4.571956,0.328903,0.250000
55,2000-04-07,ATL,SFN,3.875000,34.875000,9.625000,1.000000,0.000000,1.625000,3.125000,0.500000,0.000000,0.750000,2.875000,0.250000,5.250000,0.250000,0.125000,0.875000,9.250000,4.375000,4.375000,4.375000,0.000000,27.000000,10.750000,0.500000,0.500000,0.875000,0.0,0.339700,0.274116,7.000000,0.443056,0.282297,0.168939,6.906250,15.500000,5.133724,0.348635,0.375000,5.812500,34.500000,8.687500,1.500000,0.125000,2.375000,5.312500,0.125000,0.750000,0.000000,3.375000,0.000000,5.312500,0.125000,0.000000,0.750000,5.937500,2.875000,3.500000,3.500000,0.500000,26.562500,6.312500,0.125000,0.125000,0.812500,0.0,0.309788,0.248264,4.687500,0.503628,0.223733,0.255364,6.560937,17.562500,5.564117,0.350106,0.687500
70,2000-04-08,ATL,SFN,5.437500,33.437500,10.312500,1.000000,0.500000,1.312500,5.062500,1.250000,0.000000,0.875000,3.437500,0.125000,5.625000,0.625000,0.062500,0.437500,8.625000,3.687500,4.687500,4.687500,0.500000,27.000000,11.875000,0.250000,0.250000,1.937500,0.0,0.386066,0.308933,7.500000,0.487153,0.341149,0.178220,6.119792,16.250000,6.108528,0.384993,0.687500,5.406250,34.250000,9.343750,0.750000,0.062500,1.687500,5.156250,0.062500,0.375000,0.500000,2.187500,0.000000,5.656250,0.562500,0.000000,1.875000,4.968750,3.937500,5.250000,5.250000,0.250000,25.281250,6.656250,0.062500,0.062500,0.406250,0.0,0.321561,0.271191,6.843750,0.442990,0.278533,0.171800,6.113802,15.281250,4.824916,0.335053,0.343750
85,2000-04-09,ATL,SFN,7.218750,34.218750,11.156250,1.000000,1.250000,1.156250,6.531250,1.625000,0.500000,0.437500,4.718750,0.062500,5.312500,1.812500,0.031250,0.218750,9.812500,3.343750,3.843750,3.843750,0.250000,27.000000,10.437500,0.125000,0.125000,0.96